## MLflow's Model Registry

In [56]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [57]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location=('/Users/aasth/Desktop/Data '
  'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/2'), creation_time=1700889822238, experiment_id='2', last_update_time=1700889822238, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location=('/Users/aasth/Desktop/Data '
  'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1700705746970, experiment_id='1', last_update_time=1700705746970, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1700705573743, experiment_id='0', last_update_time=1700705573743, lifecycle_stage='active', name='Default', tags={}>]

In [58]:
try:
    client.create_experiment(name="my-cool-experiment")
except:
    pass

Let's check the latest versions for the experiment with id `1`...

In [63]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 10",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [64]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f459d99590f9460295ea6d82762a2a03, rmse: 4.9809
run id: 3f259ee75a824c2fb5051229d9bb2bb2, rmse: 5.0170
run id: 9db8170c993147c0ba35aa197f032abc, rmse: 5.0592
run id: d9a5d41f07ef41f4a8eefa27bfe46b07, rmse: 5.1146
run id: 2dabc4596bf64f43a0590490745b0597, rmse: 5.6476


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [73]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [87]:
run_id = "2dabc4596bf64f43a0590490745b0597"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor-final")

Registered model 'nyc-taxi-regressor-final' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor-final'.


<ModelVersion: aliases=[], creation_timestamp=1701136692524, current_stage='None', description=None, last_updated_timestamp=1701136692524, name='nyc-taxi-regressor-final', run_id='2dabc4596bf64f43a0590490745b0597', run_link=None, source=('/Users/aasth/Desktop/Data '
 'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dabc4596bf64f43a0590490745b0597/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [75]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1700890045698, description=None, last_updated_timestamp=1701135644803, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1700890045715, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2023-11-25', last_updated_timestamp=1700890355697, name='nyc-taxi-regressor', run_id='2dabc4596bf64f43a0590490745b0597', run_link=None, source=('/Users/aasth/Desktop/Data '
  'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dabc4596bf64f43a0590490745b0597/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>,
  <ModelVersion: aliases=[], creation_timestamp=1701131732773, current_stage='Archived', description='The model version 2 was transitioned to Staging on 2023-11-27', last_updated_timestamp=1701135644803, name='nyc-taxi-regressor', run_id='f459d99590f9460295ea6d82762a2a03', run_link=None, source=('/Users/aasth/Desktop/Data '
  'a

In [88]:
model_name = "nyc-taxi-regressor-final"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Archived
version: 5, stage: None
version: 4, stage: Staging


In [89]:
#moving xgboost to staging
model_version = 5
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1701136797739, current_stage='Staging', description='', last_updated_timestamp=1701136837805, name='nyc-taxi-regressor-final', run_id='2dabc4596bf64f43a0590490745b0597', run_link='', source=('/Users/aasth/Desktop/Data '
 'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dabc4596bf64f43a0590490745b0597/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'XGBoost'}, user_id=None, version=5>

In [90]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1701136797739, current_stage='Staging', description='The model version 5 was transitioned to Staging on 2023-11-27', last_updated_timestamp=1701136839888, name='nyc-taxi-regressor-final', run_id='2dabc4596bf64f43a0590490745b0597', run_link='', source=('/Users/aasth/Desktop/Data '
 'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dabc4596bf64f43a0590490745b0597/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'XGBoost'}, user_id=None, version=5>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [79]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [80]:
df = read_dataframe("/Users/aasth/Desktop/Data analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/data/yellow_tripdata_2022-03.parquet")

In [81]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/aasth/Desktop/Data analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [82]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [83]:
X_test = preprocess(df, dv)

In [84]:
target = "duration"
y_test = df[target].values

In [91]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:00:55] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 35 s, sys: 698 ms, total: 35.7 s
Wall time: 5.33 s


{'rmse': 5.942320674583259}

In [92]:
client.transition_model_version_stage(
    name=model_name,
    version=5,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1701136797739, current_stage='Production', description='The model version 5 was transitioned to Staging on 2023-11-27', last_updated_timestamp=1701136914410, name='nyc-taxi-regressor-final', run_id='2dabc4596bf64f43a0590490745b0597', run_link='', source=('/Users/aasth/Desktop/Data '
 'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dabc4596bf64f43a0590490745b0597/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'XGBoost'}, user_id=None, version=5>

In [93]:
client.update_model_version(
    name=model_name,
    version=5,
    description=f"The model version 5 was transitioned to Production on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1701136797739, current_stage='Production', description='The model version 5 was transitioned to Production on 2023-11-27', last_updated_timestamp=1701136921512, name='nyc-taxi-regressor-final', run_id='2dabc4596bf64f43a0590490745b0597', run_link='', source=('/Users/aasth/Desktop/Data '
 'analytics/MLOps/datatalks-zoomcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/2dabc4596bf64f43a0590490745b0597/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'XGBoost'}, user_id=None, version=5>

In [94]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test) #xgboost time

/opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:02:51] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 34.5 s, sys: 581 ms, total: 35.1 s
Wall time: 4.77 s


{'rmse': 5.942320674583259}

In [95]:
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Archived
version: 3, stage: None
version: 4, stage: Staging
version: 5, stage: Production


In [96]:
#linear regression time
# To fetch a model version by stage, simply provide the model stage as part of the model URI, and it will fetch the most recent version of the model in that stage.
# clearer to fetch a model by its version number than stage but in this case we are using stage 
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 74.2 ms, sys: 90.8 ms, total: 165 ms
Wall time: 299 ms


{'rmse': 9.913258959342382}